In [1]:
import pandas as pd
import numpy as np

import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm
import gc
import re
from sklearn.metrics import roc_auc_score
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import Word2Vec
import polars as pl
from pathlib import Path
from glob import glob
import json
import joblib

In [4]:
import os
try:
    os.makedirs( 'data')
except:
    continue
try:
    os.makedirs('temp_data')
except:
    continue
try:
    os.makedirs('sub')
except:
    continue
try:
    os.makedirs('feats/w2v_feats')
except:
    continue
try:
    os.makedirs('bge_m3_emb/w2v_feats')
except:
    continue
try:
    os.makedirs('oof2')
except:
    continue



In [2]:
with open('../IND-WhoIsWho/pid_to_info_all.json', 'r') as file:
    data = json.load(file)


pid_df = []
for f in tqdm(data.keys()):
    
    temp = list(data[f].values())
    temp = pd.DataFrame([temp],columns = data[f].keys())
    pid_df.append(temp.copy())
pid_df = pd.concat(pid_df)
joblib.dump(pid_df,'data/pid_df.pkl')


100%|██████████| 317302/317302 [02:15<00:00, 2348.13it/s]


['data/pid_df.pkl']

In [3]:
with open('../IND-test-public/ind_test_author_filter_public.json', 'r') as file:
    data = json.load(file)
    
test = []
for key in data.keys():
    temp = pd.DataFrame(data[key]['papers'],columns = ['PID'])
    temp['autherID'] = key
    temp['autherName'] = data[key]['name']
    
    test.append(temp.copy())
test = pd.concat(test).reset_index(drop = True)
test.to_feather('data/test.feather')

In [4]:

import json
 
with open('../IND-WhoIsWho/train_author.json', 'r') as file:
    data = json.load(file)
    
train = []
for key in data.keys():
    temp1 = pd.DataFrame(data[key]['normal_data'],columns = ['PID'])
    temp1['label'] = 0 


    temp2 = pd.DataFrame(data[key]['outliers'],columns = ['PID'])
    temp2['label'] = 1
    temp = pd.concat([temp1,temp2]).reset_index(drop = True)
    temp['autherID'] = key
    temp['autherName'] = data[key]['name']
    train.append(temp.copy())
train = pd.concat(train).reset_index(drop = True)
train.to_feather('data/train.feather')

In [5]:
with open('../IND-WhoIsWho/ind_valid_author.json', 'r') as file:
    data = json.load(file)
    
test = []
for key in data.keys():
    temp = pd.DataFrame(data[key]['papers'],columns = ['PID'])
    temp['autherID'] = key
    temp['autherName'] = data[key]['name']
    
    test.append(temp.copy())
test = pd.concat(test).reset_index(drop = True)
test.to_feather('data/valid.feather')